In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/data1.csv', header=None)
print(data)
labels=pd.read_csv('C:/Users/bhatt/OneDrive/Desktop/whisper-main/ANN_Data/label1.csv',header=None)
print(labels)

             0         1         2         3         4         5         6   \
0     -0.446891 -0.013397  0.232645  2.156649  1.652923 -0.210531 -0.662227   
1      0.243747 -1.144175 -0.622214 -0.661979 -0.373315  0.520313 -1.307954   
2     -1.417799 -0.088833 -0.647181 -1.141497 -1.321100  0.686798 -1.045569   
3      0.656784  0.381201 -1.039011  1.285315  0.808324  1.428519 -1.144367   
4      1.098520  1.706769 -1.030370  2.001009  2.751551  1.202229 -0.941066   
...         ...       ...       ...       ...       ...       ...       ...   
19995  0.146324 -0.618555  0.547413  0.851301  0.838880 -1.248402  0.698199   
19996 -2.227923 -0.327196  1.258849 -0.945254  3.925262  0.316686 -1.061159   
19997 -0.800405  1.477997  0.090057 -0.973997 -0.204970  0.224539  0.302265   
19998 -1.217344 -0.785401 -0.413781 -0.335324  0.181671 -1.685733 -0.437981   
19999  1.601005 -0.890398 -1.425049  0.887501  0.167205 -0.395042 -0.512554   

             7         8         9         10      

In [3]:
data=data.values
npLabels=labels.values
print(data.shape)
print(npLabels.shape)

(20000, 20)
(20000, 1)


In [4]:
print(np.max(npLabels))
print(np.min(npLabels))

4
0


In [5]:
np.unique(npLabels)

array([0, 1, 2, 3, 4], dtype=int64)

In [6]:
num=np.max(npLabels)+1
oneHot=np.eye(num)[npLabels]
print(oneHot.ndim)
print(oneHot.shape)

3
(20000, 1, 5)


In [7]:
oneHotRe=oneHot.reshape(20000, 5)
print(oneHotRe)

[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [8]:
trainData,testDataDash=train_test_split(data,train_size=0.8,test_size=0.2,shuffle=False)
trainLabel,testLabelDash=train_test_split(oneHotRe,train_size=0.8,test_size=0.2,shuffle=False)

In [9]:
validData,testData=train_test_split(testDataDash,train_size=0.5,test_size=0.5,shuffle=False)
validLabel,testLabel=train_test_split(testLabelDash,train_size=0.5,test_size=0.5,shuffle=False)

In [10]:
h = 15

In [11]:
b1=np.zeros(h)
b2=np.zeros(5)
print(b1)
print(b2)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [12]:
w1=np.random.normal(0, 1, (20,h))
w2=np.random.normal(0, 1, (h,5))

In [13]:
def actFunc(huehue,choice):
    #tanh
    if(choice==1):
        return np.tanh(huehue)
    elif(choice==2):
        temp=np.exp(huehue)
        return temp/np.sum(temp,axis=1,keepdims=True)

In [14]:
alpha = 0.003

In [15]:
batch=1000

In [16]:
def costFunc(y,t):
    return -(t*(np.log(y))+(1-t)*np.log(1-y))

In [17]:
def forwFunc(data):
    a0=data
    z1=np.dot(a0,w1)+b1
    a1=actFunc(z1,1)
    z2=np.dot(a1,w2)+b2
    a2=actFunc(z2,2)
    return a0,z1,a1,z2,a2

In [18]:
def Acc(y,t,size):
    maxData=np.argmax(y,axis=1)
    maxLabel=np.argmax(t,axis=1)
    compare=np.equal(maxData,maxLabel)
    count=np.sum(compare)
    return (count/size)*100

In [19]:
epochs=500

In [20]:
for epoch in range(epochs):
    for iteration in range(int(len(trainData)/batch)):
        a0,z1,a1,z2,a2=forwFunc(trainData[iteration*batch:(iteration+1)*batch,:])
        y=a2
        labelBatch=trainLabel[iteration*batch:(iteration+1)*batch,:]
        del2=(y-labelBatch)
        del1=np.dot(del2,w2.T)*(1 - a1**2)
        dcdw2=np.dot(a1.T,del2)
        dcdw1=np.dot(a0.T,del1)
        dcdb1=np.sum(del1,axis=0)
        dcdb2=np.sum(del2,axis=0)
        w1=w1-alpha*dcdw1
        w2=w2-alpha*dcdw2
        b2=b2-alpha*dcdb2
        b1=b1-alpha*dcdb1
    a0,z1,a1,z2,a2=forwFunc(trainData)
    print("Training Cost",(np.sum(costFunc(a2,trainLabel)))/16000.0)
    print("Training accuracy",Acc(a2,trainLabel,len(trainLabel)))
    
va0,vz1,va1,vz2,va2=forwFunc(validData)
vOutput=va2
accuracy = Acc(vOutput,validLabel,2000.0)
print("validation accuracy")
print(accuracy)

Training Cost 1.3773545561409082
Training accuracy 71.63125000000001
Training Cost 1.1116407286806713
Training accuracy 77.09375
Training Cost 1.0331734917004625
Training accuracy 78.825
Training Cost 0.9701411098509127
Training accuracy 80.36875
Training Cost 0.9072478144036833
Training accuracy 81.975
Training Cost 0.8433830574374541
Training accuracy 83.76249999999999
Training Cost 0.7993934963268944
Training accuracy 84.99375
Training Cost 0.7770842172539805
Training accuracy 85.5125
Training Cost 0.7634092999535171
Training accuracy 85.90625
Training Cost 0.7548196997772556
Training accuracy 85.98125
Training Cost 0.7425581496047667
Training accuracy 86.14375
Training Cost 0.726482268321033
Training accuracy 86.36875
Training Cost 0.7122071611371709
Training accuracy 86.5375
Training Cost 0.7001188364228623
Training accuracy 86.6875
Training Cost 0.6911596902369608
Training accuracy 86.725
Training Cost 0.6816309238089105
Training accuracy 86.93125
Training Cost 0.6731715790842218

Training Cost 0.4734037245104909
Training accuracy 91.5125
Training Cost 0.47334456188115337
Training accuracy 91.50625
Training Cost 0.4732902845572645
Training accuracy 91.4875
Training Cost 0.4732401134238301
Training accuracy 91.475
Training Cost 0.4731933420875595
Training accuracy 91.4625
Training Cost 0.47314934274493364
Training accuracy 91.48125
Training Cost 0.47310756629311473
Training accuracy 91.50625
Training Cost 0.4730675386986804
Training accuracy 91.49374999999999
Training Cost 0.47302885521125526
Training accuracy 91.49374999999999
Training Cost 0.4729911735821934
Training accuracy 91.48125
Training Cost 0.4729542070768133
Training accuracy 91.48125
Training Cost 0.472917717769366
Training accuracy 91.4875
Training Cost 0.4728815103823697
Training accuracy 91.48125
Training Cost 0.47284542676876407
Training accuracy 91.4875
Training Cost 0.4728093410272217
Training accuracy 91.5
Training Cost 0.4727731551785357
Training accuracy 91.5
Training Cost 0.47273679530501433

Training Cost 0.4687809313497469
Training accuracy 91.71249999999999
Training Cost 0.46876283550145553
Training accuracy 91.71875
Training Cost 0.4687447844512603
Training accuracy 91.71249999999999
Training Cost 0.4687267903345928
Training accuracy 91.71249999999999
Training Cost 0.4687088675590624
Training accuracy 91.70625
Training Cost 0.46869103232803516
Training accuracy 91.70625
Training Cost 0.4686733020213186
Training accuracy 91.70625
Training Cost 0.46865569446364647
Training accuracy 91.7
Training Cost 0.46863822712872794
Training accuracy 91.7
Training Cost 0.4686209163391677
Training accuracy 91.7
Training Cost 0.46860377652759566
Training accuracy 91.69375
Training Cost 0.4685868196201783
Training accuracy 91.7
Training Cost 0.4685700545904953
Training accuracy 91.71249999999999
Training Cost 0.4685534872117521
Training accuracy 91.71875
Training Cost 0.4685371200120902
Training accuracy 91.7
Training Cost 0.46852095241552
Training accuracy 91.69375
Training Cost 0.46850

Training Cost 0.4659120680505702
Training accuracy 91.725
Training Cost 0.46588579057341994
Training accuracy 91.725
Training Cost 0.4658592970329495
Training accuracy 91.725
Training Cost 0.465832588364438
Training accuracy 91.74375
Training Cost 0.46580566512677435
Training accuracy 91.74375
Training Cost 0.46577852747697857
Training accuracy 91.74375
Training Cost 0.4657511752002933
Training accuracy 91.73125
Training Cost 0.4657236077970175
Training accuracy 91.73125
Training Cost 0.4656958246235728
Training accuracy 91.73125
Training Cost 0.46566782508136007
Training accuracy 91.73125
Training Cost 0.4656396088429738
Training accuracy 91.725
Training Cost 0.46561117610147534
Training accuracy 91.73125
Training Cost 0.46558252782495607
Training accuracy 91.73125
Training Cost 0.46555366599578135
Training accuracy 91.73125
Training Cost 0.46552459381203226
Training accuracy 91.73750000000001
Training Cost 0.4654953158280806
Training accuracy 91.73750000000001
Training Cost 0.4654658